## SVM Model

In [1]:
print("Importing Libraries...")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
import time
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import math
from sklearn.metrics import r2_score
print("Successfully imported!")

Importing Libraries...
Successfully imported!


In [2]:
print("Reading new dataset...")
checkpoint6 = pd.read_csv('D:\ML Files\checkpoint-6-combined.csv', nrows=300000)
print("Success!")

Reading new dataset...
Success!


In [17]:
checkpoint6.head(20)
checkpoint6.isnull().any()

passenger_count    False
day                False
weekend            False
holiday            False
peak_hour          False
hotspot            False
dist               False
date               False
year               False
month              False
dtype: bool

In [4]:
print("Obtaining label...")
cols = 'fare_amount'
label = checkpoint6[[cols]]
label
print("Obtained labels!")

Obtaining label...
Obtained labels!


In [5]:
# del checkpoint5['dist']
del checkpoint6['time']
del checkpoint6['fare_amount']
del checkpoint6['pickup_longitude']
del checkpoint6['pickup_latitude']
del checkpoint6['dropoff_longitude']
del checkpoint6['dropoff_latitude']

In [6]:
checkpoint6.head(20)

,passenger_count,day,weekend,holiday,peak_hour,hotspot,dist,date,year,month
0,1,0,False,0,1,0,1.030764,15,2009,6
1,1,1,False,0,1,0,8.450134,5,2010,1
2,2,3,False,0,0,1,1.389525,18,2011,8
3,1,5,True,0,0,1,2.799270,21,2012,4
4,1,1,False,0,1,0,1.999157,9,2010,3
5,1,3,False,0,0,0,3.787239,6,2011,1
6,1,1,False,0,0,0,1.555807,20,2012,11
7,1,2,False,0,1,0,4.155444,4,2012,1
8,1,0,False,0,0,1,1.253232,3,2012,12
9,2,2,False,0,0,0,2.849627,2,2009,9


In [7]:
print("Dropped columns!")

Dropped columns!


In [8]:
# Splitting Dataset into train and test (20% TEST, 80% TRAIN) 
print('Splitting Dataset...')
X_train, X_test, y_train, y_test = train_test_split(checkpoint6, label, test_size = 0.2)
print("Dataset splitted!")

Splitting Dataset...
Dataset splitted!


### Normalising the values

In [9]:
X_sc = StandardScaler()
y_sc = StandardScaler()

In [10]:
X_train = X_sc.fit_transform(X_train)
y_train = y_sc.fit_transform(y_train)

In [11]:
print('Fitting the model...')
regressor = SVR(kernel = 'rbf')
t0 = time.time()
regressor.fit(X_train, y_train.ravel())
t1 = time.time()
print('Succeeded in fitting model!')

Fitting the model...
Succeeded in fitting model!


In [12]:
print("Saving model...")

with open('WithDistDate_SVM/Dist_SVM_Experiment_300000.pkl', 'wb') as f:

   pickle.dump(regressor, f)

print("Saved model!")

Saving model...
Saved model!


In [13]:
print('Predicting...')
y_pred = regressor.predict(X_sc.transform(X_test))
t2 = time.time()
y_pred = y_sc.inverse_transform(y_pred.reshape(-1, 1))
print('Predicted all!')
print(y_pred)

Predicting...
Predicted all!
[[ 7.45018509]
 [ 7.04696732]
 [10.93941609]
 ...
 [ 7.17723503]
 [ 6.13532578]
 [46.58364754]]


In [14]:
y_test = y_test.to_numpy()

In [15]:
y_test = y_test.flatten()
y_pred = y_pred.flatten()
y_test_series = pd.Series(y_test)
y_pred_series = pd.Series(y_pred)
df = pd.DataFrame({'Predicted value': y_pred, 'Real Value': y_test})
df
df.to_csv('WithDistDate_SVM/Dist_predictions_SVR300000.csv',index=False )

In [16]:
time_train = t1-t0
time_predict = t2-t1

#Get Report
print("Score for classifier:", r2_score(y_test,y_pred))
print ("Time taken to train:", time_train)
print("Time taken to predict",time_predict)
print("Mean square error is -> ",mean_squared_error(y_test, y_pred))
print("Root Mean square error is -> ",math.sqrt(mean_squared_error(y_test, y_pred)))
print("Mean absolute error is -> ",mean_absolute_error(y_test, y_pred))

# Save report 
report = { 
    'version': 'SVM with Dist Date',
    'number_rows': 300000,
    'train time':time_train,
    'r2_score': r2_score(y_test,y_pred),
    'MSE': mean_squared_error(y_test, y_pred),
    'RMSE': math.sqrt(mean_squared_error(y_test, y_pred)),
    'MAE': mean_absolute_error(y_test, y_pred)}

final_report = pd.DataFrame.from_dict(report.items())
final_report.to_csv('WithDistDate_SVM/report_SVR300000.csv',index=False )

Score for classifier: 0.6100008663651038
Time taken to train: 4431.89809012413
Time taken to predict 338.98486495018005
Mean square error is ->  39.38945560613004
Root Mean square error is ->  6.276101943573737
Mean absolute error is ->  2.31408394406531


In [17]:
# with open('SVM_Experiment1.pkl', 'rb') as f:

#    regressor = pickle.load(f)